In [2]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer

def tangoretu(x):
    sentence=[]
    x=x.lower() 
    x=re.sub("[0-9]+","",x)
    x=re.sub("[^0a-zA-Z ]"," ",x)
    words=x.split()
    for word in words:
        sentence.append(word)
    return " ".join(sentence)

def write_tangoretu(df,write_text):
    df["TITLE"]=df["TITLE"].map(lambda x:tangoretu(x))
    return df

df_train=pd.read_table("train.txt",header=None,names=["CATEGORY","TITLE"])
write_tangoretu(df_train,"train.feature.txt")

df_valid=pd.read_table("valid.txt",header=None,names=["CATEGORY","TITLE"])
write_tangoretu(df_valid,"valid.feature.txt")

df_test=pd.read_table("test.txt",header=None,names=["CATEGORY","TITLE"])
write_tangoretu(df_test,"test.feature.txt")

df_m=pd.concat([df_train,df_valid,df_test],axis=0)

train_valid= df_m[:len(df_train) + len(df_valid)]
test = df_m[len(df_train) + len(df_valid):]

vec_tfidf = TfidfVectorizer(min_df=15, ngram_range=(1,1))

X_m=vec_tfidf.fit_transform(train_valid['TITLE'])
X_test = vec_tfidf.transform(test['TITLE'])


X_m=pd.DataFrame(X_m.toarray(), columns=vec_tfidf.get_feature_names())
X_test = pd.DataFrame(X_test.toarray(), columns=vec_tfidf.get_feature_names())

X_train=X_m[:len(df_train)]
X_valid= X_m[len(df_train):]

X_train.to_csv("X_train.txt",sep="\t",index=False)
X_valid.to_csv("X_valid.txt", sep="\t", index=False)
X_test.to_csv("X_test.txt", sep="\t", index=False)